# Project EdgeAI

## Import library to get data

In [5]:
import requests, os, shutil

listQuery = ["Emberiza citrinella", "Alauda arvensis", "Emberiza cirlus", "Cuculus canorus"] #, "Tyto alba", "Falco tinnunculus", "Sylvia borin", "Muscicapa striata", "Tachybaptus ruficollis", "Delichon urbicum"]

## Import data from API

In [ ]:
url = "https://xeno-canto.org/api/2/recordings?query="

def enregistrer_mp3(url_api, nom_fichier):
    reponse = requests.get(url_api, stream=True)
    
    if reponse.status_code == 200:
        with open(nom_fichier, 'wb') as fichier_audio:
            reponse.raw.decode_content = True
            shutil.copyfileobj(reponse.raw, fichier_audio)
        print("Audio enregistré avec succès sous le nom :", nom_fichier)
    else:
        print("Erreur lors de la récupération des données audio depuis l'API")

def getData(query):
    url_api = url + query
    response = requests.get(url_api)

    if response.status_code == 200:
        contenu = response.json()["recordings"]
        pathToDir = "./data/" + query

        if not os.path.exists(pathToDir):
            os.mkdir(pathToDir)

            for _, element in zip(range(250), contenu):
                elementUrl = element["file"]
                elementFileName = element["file-name"]
                storeFile = pathToDir + "/" + elementFileName
                
                if not os.path.exists(storeFile):
                    try:
                        enregistrer_mp3(elementUrl, storeFile)
                    except: # Pour les problèmes de noms de fichiers
                        pass

for query in listQuery:
    getData(query)

## Import library to train model

In [17]:
from sklearn import model_selection
from pathlib import Path
import librosa
import copy
import matplotlib
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Input, Conv1D, MaxPool1D, Flatten, Dense, AvgPool1D, Activation
from keras.utils import get_file
from keras.activations import softmax
from keras.utils import to_categorical

dataset_dir = Path('data')

## Load all sound in X and Y

In [6]:
x = []
y = []

def slice_audio_with_trim(file_path, index, duration=1):
    audio_data, sampling_rate = librosa.load(file_path, sr=None)
    non_silent_audio, _ = librosa.effects.trim(audio_data)
    num_slices = int(len(non_silent_audio) / sampling_rate / duration)

    # Diviser l'audio en fonction de duration
    for i in range(num_slices):
        start = i * sampling_rate * duration
        end = start + sampling_rate * duration
        slice_data = non_silent_audio[start:end]

        data = slice_data.astype(np.float32)
        data.resize((16000, 1))

        x.append(data)
        y.append(index)

def getAllDataClasses(directory, index):
    for recording in dataset_dir.glob(f'{directory}/*'):
        slice_audio_with_trim(str(recording) ,index)

def loadAllData():
    for i in range(len(listQuery)):
        getAllDataClasses(listQuery[i], i)

loadAllData()

In [7]:
u, indexes, counts = np.unique(y, return_index=True, return_counts=True)
min = np.min(counts)
nbrClasses = len(u)

indices = np.random.permutation(min)

newX = []
newY = []

for i in range(nbrClasses):
    sousSetX = []
    sousSetY = []
    
    if i == nbrClasses - 1:
        sousSetX = x[indexes[i]:]
        sousSetY = y[indexes[i]:]
    else:
        sousSetX = x[indexes[i] : indexes[i + 1]]
        sousSetY = y[indexes[i] : indexes[i + 1]]
    
    x_selectionne = [sousSetX[j] for j in indices]
    y_selectionne = [sousSetY[j] for j in indices]

    for x_s, y_s in zip(x_selectionne, y_selectionne):
        newX.append(x_s)
        newY.append(y_s)

u, indexes, counts = np.unique(newY, return_index=True, return_counts=True)
print(u)
print(indexes)
print(counts)

[0 1 2 3]
[   0 1121 2242 3363]
[1121 1121 1121 1121]


In [8]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(newX, newY, test_size=0.2)
x_train = np.array(x_train)
x_test = np.array(x_test)
y_train = to_categorical(np.array(y_train))
y_test = to_categorical(np.array(y_test))


In [9]:
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)

(3587, 16000, 1) (897, 16000, 1)
(3587, 4) (897, 4)


In [121]:
model = Sequential()
model.add(Input(shape=(16000, 1)))
model.add(MaxPool1D(pool_size=2))
model.add(Conv1D(filters=16, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=4))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=8))
model.add(AvgPool1D())
model.add(Conv1D(filters=4, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=8))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPool1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=4))
model.add(Activation('softmax')) # SoftMax activation needs to be separate from Dense to remove it later on
# EXPLORE Learning Rate
opt = tf.keras.optimizers.Adam(learning_rate=10e-4)
model.summary()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

Model: "sequential_66"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 max_pooling1d_256 (MaxPooli  (None, 8000, 1)          0         
 ng1D)                                                           
                                                                 
 conv1d_244 (Conv1D)         (None, 7998, 16)          64        
                                                                 
 max_pooling1d_257 (MaxPooli  (None, 1999, 16)         0         
 ng1D)                                                           
                                                                 
 conv1d_245 (Conv1D)         (None, 1997, 32)          1568      
                                                                 
 max_pooling1d_258 (MaxPooli  (None, 249, 32)          0         
 ng1D)                                                           
                                                     

In [122]:
model.fit(x_train, y_train, epochs=50, batch_size=10, validation_data=(x_test, y_test))
# model.fit(x_train, y_train, epochs=25, batch_size=268, validation_data=(x_test, y_test))

Epoch 1/50
359/359 [==============================] - 7s 17ms/step - loss: 1.3322 - categorical_accuracy: 0.3412 - val_loss: 1.0644 - val_categorical_accuracy: 0.5184
Epoch 2/50
359/359 [==============================] - 6s 15ms/step - loss: 0.8793 - categorical_accuracy: 0.5768 - val_loss: 0.7894 - val_categorical_accuracy: 0.6388
Epoch 3/50
359/359 [==============================] - 6s 15ms/step - loss: 0.7718 - categorical_accuracy: 0.6295 - val_loss: 0.7066 - val_categorical_accuracy: 0.6656
Epoch 4/50
359/359 [==============================] - 6s 17ms/step - loss: 0.7054 - categorical_accuracy: 0.6657 - val_loss: 0.7292 - val_categorical_accuracy: 0.6778
Epoch 5/50
359/359 [==============================] - 6s 17ms/step - loss: 0.6785 - categorical_accuracy: 0.6875 - val_loss: 0.6593 - val_categorical_accuracy: 0.7280
Epoch 6/50
359/359 [==============================] - 6s 15ms/step - loss: 0.6595 - categorical_accuracy: 0.7003 - val_loss: 0.7861 - val_categorical_accuracy: 0.675

In [123]:
model.evaluate(x_test, y_test, verbose=2)
pred_test = model.predict(x_test)
print(tf.math.confusion_matrix(y_test.argmax(axis=1), pred_test.argmax(axis=1)))

29/29 - 0s - loss: 0.6440 - categorical_accuracy: 0.7949 - 394ms/epoch - 14ms/step
29/29 [==============================] - 0s 13ms/step
tf.Tensor(
[[194   4   9   5]
 [  3 203  16   0]
 [106  11 112   1]
 [ 23   0   6 204]], shape=(4, 4), dtype=int32)


In [69]:
model.save('project.h5')

In [70]:
if isinstance(model.layers[-1], Activation) and model.layers[-1].activation == softmax:
    model = tf.keras.Model(model.input, model.layers[-2].output, name=model.name)
else:
    print('Error: last layer is not SoftMax Activation')

In [71]:
%pip install qualia_codegen_core
import qualia_codegen_core
from qualia_codegen_core.graph.KerasModelGraph import KerasModelGraph
from qualia_codegen_core.graph.Quantization import Quantization
from qualia_codegen_core.graph.RoundMode import RoundMode

from importlib.resources import files
main_path = str((files('qualia_codegen_core.examples')/'Linux'/'main.cpp').resolve())

Note: you may need to restart the kernel to use updated packages.


In [20]:
modelgraph = KerasModelGraph(model).convert()
print(modelgraph)

—————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
Inputs                                           | Layer                                            | Outputs                                          | Input shape                                      | Output shape                                    
—————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————————
                                                 | input_1                                          | conv1d                                           | (1, 16000, 1)                                    | ((1, 16000, 1),)                   

In [21]:
float_modelgraph = copy.deepcopy(modelgraph)

# layer quantization annotations for float32
for node in float_modelgraph.nodes:
    # No scale factor if not fixed-point quantization on integers
    node.q = Quantization(
            number_type=float,
            width=32,
            long_width=32,
            weights_scale_factor=0,
            output_scale_factor=0,
            weights_round_mode=RoundMode.NONE,
            output_round_mode=RoundMode.NONE,
            )

float_res = qualia_codegen_core.Converter(output_path=Path('gsc_output_floating')).convert_model(float_modelgraph)

with open('gsc_model_floating.h', 'w') as f:
    f.write(float_res)

Keras weights file (<HDF5 file "variables.h5" (mode r+)>) saving:
...layers\conv1d
......vars
.........0
.........1
...layers\conv1d_1
......vars
.........0
.........1
...layers\dense
......vars
.........0
.........1
...layers\flatten
......vars
...layers\input_layer
......vars
...layers\max_pooling1d
......vars
...layers\max_pooling1d_1
......vars
...vars
Keras model archive saving:
File Name                                             Modified             Size
config.json                                    2024-04-22 22:47:08         2894
metadata.json                                  2024-04-22 22:47:08           64
variables.h5                                   2024-04-22 22:47:08        82616
Keras model archive loading:
File Name                                             Modified             Size
config.json                                    2024-04-22 22:47:08         2894
metadata.json                                  2024-04-22 22:47:08           64
variables.h5            

Graphviz not available


In [23]:
!g++ -std=c++17 -Wall -Wextra -pedantic -Ofast -o gsc_fixed -include gsc_output_floating/include/defines.h -Igsc_output_floating/include gsc_output_floating/model.c {main_path}

In file included from gsc_output_floating/model.c:15:
gsc_output_floating/include/number.h: In function 'float scale_number_t_float(float, int, round_mode_t)':
gsc_output_floating/include/number.h:73:21: warning: unused parameter 'scale_factor' [-Wunused-parameter]
   73 |   float number, int scale_factor, round_mode_t round_mode) {
      |                 ~~~~^~~~~~~~~~~~
gsc_output_floating/include/number.h:73:48: warning: unused parameter 'round_mode' [-Wunused-parameter]
   73 |   float number, int scale_factor, round_mode_t round_mode) {
      |                                   ~~~~~~~~~~~~~^~~~~~~~~~
gsc_output_floating/include/number.h: In function 'float scale_and_clamp_to_number_t_float(float, int, round_mode_t)':
gsc_output_floating/include/number.h:81:21: warning: unused parameter 'scale_factor' [-Wunused-parameter]
   81 |   float number, int scale_factor, round_mode_t round_mode) {
      |                 ~~~~^~~~~~~~~~~~
gsc_output_floating/include/number.h:81:48: warnin